In [29]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split as surprise_train_test_split
from sklearn.metrics.pairwise import cosine_similarity


from surprise import accuracy, Reader, Dataset
from surprise import NormalPredictor, KNNBasic, SVD, SVDpp, CoClustering, SlopeOne, NMF, KNNBaseline
from surprise.model_selection import cross_validate, KFold, GridSearchCV, train_test_split


#### **5. Modeling : Recommendation System**

The recommendation will be in three different approach : content-based method, collaborative filtering and combining both in a hybrid model of recommendation system.

The difference in recommendations between content-based filtering and collaborative filtering comes from the underlying mechanisms of how these methods work:

**A. Content-Based Filtering:**

**How it works:** Content-based filtering recommends items that are similar to those a user has liked in the past, based on the features of the items themselves. In this case, features like product_category_name_english are used to find similarities between items.

Since the method is based on features (like category, description length, etc.), it will naturally recommend items that are similar to those the user has interacted with in the same category. For example, if the user likes electronics, content-based filtering will recommend other electronics products, as they share similar attributes (e.g., category).

**In-Short :** Content-Based Filtering is based solely on the similarity of item features.

**B. Collaborative Filtering:**

**How it works:** Collaborative filtering uses the preferences of many users to make recommendations. It looks at what other users with similar tastes have liked and recommends those items to you, regardless of the content or category of the items.

Collaborative filtering does not directly consider item features like categories. Instead, it looks at patterns of user behavior. If other users who liked the same items as you have also liked items in different categories, it may recommend those cross-category items to you. This is why collaborative filtering can recommend items from different categories—because it leverages user behavior patterns, not just item similarities.

**In-Short :** Collaborative Filtering considers user preferences based on their ratings of other items.

##### **5.1 Feature Engineering and Selection**

In [30]:
df_olist = pd.read_csv('./dataset/df_olist_clean.csv')

df_olist['order_purchase_timestamp'] = pd.to_datetime(df_olist['order_purchase_timestamp'])

    5.1.1 Creating New Features

In [31]:
# Calculate seller average review score
df_olist['seller_avg_review_score'] = df_olist.groupby('seller_id')['review_score'].transform('mean')

    5.1.2 Dataset Preparation : Model Performance Evaluation

In this section, we will separate the dataset to be used for building and testing models by dividing it between customers who have made their first purchase, which will later be used for a content-based filtering model and a hybrid recommendation system, and customers who have made their second purchase, which will be used for a collaborative filtering model.

In [32]:
customer_order_counts = df_olist.groupby('customer_unique_id')['order_id'].nunique()

repeat_customers_ids = customer_order_counts[customer_order_counts > 1].index


repeat_buyers = df_olist[df_olist['customer_unique_id'].isin(repeat_customers_ids)]

    5.1.2.A Repeat Buyers Two Transaction

In [33]:
# 1. Ensure the data contains only customers with exactly 2 transactions
repeat_buyers_two_transaction = repeat_buyers.groupby('customer_unique_id').filter(lambda x: len(x) == 2)

# 2. Get the last transaction of each unique customer
last_transaction = repeat_buyers_two_transaction.sort_values(['customer_unique_id', 'order_id']).groupby('customer_unique_id').tail(1).reset_index(drop=True)

# 3. Select the first 100 unique customers' last transactions
repeat_buyers_same_category = last_transaction.head(100).reset_index(drop=True)

In [34]:
repeat_buyers_same_category.shape 

(100, 27)

    5.1.2.B Repeat Buyers Different Categories

In [35]:
# 1.
different_category = repeat_buyers_two_transaction.groupby('customer_unique_id').filter(
    lambda x: x.iloc[0]['product_category_name'] != x.iloc[1]['product_category_name']
)

# 2.
different_category_transaction = different_category.groupby('customer_unique_id').tail(1).reset_index(drop=True)


# 3.
repeat_buyers_diff_category = different_category_transaction.head(100).reset_index(drop=True)

In [36]:
repeat_buyers_diff_category.shape

(100, 27)

    5.1.3 Dataset Preparation : Content-Based Filtering & Hybrid Model

In [37]:
df_olist.shape

(114085, 27)

In [38]:
# 2.
df_olist = df_olist.merge(repeat_buyers_same_category[['customer_unique_id', 'order_id']], on=['customer_unique_id', 'order_id'], how='left', indicator=True)

# 3.
df_olist = df_olist[df_olist['_merge'] == 'left_only'].drop('_merge', axis=1)

# 4. 
df_olist = df_olist.merge(repeat_buyers_diff_category[['customer_unique_id', 'order_id']], on=['customer_unique_id', 'order_id'], how='left', indicator=True)


# 5.
df_olist = df_olist[df_olist['_merge'] == 'left_only'].drop('_merge', axis=1)

In [39]:
# 7. Last check the final dataset row for modeling
df_olist.shape

(113889, 27)

In [40]:
df_content =  df_olist[['product_id','product_category_name']].drop_duplicates()
df_hybrid = df_olist.copy()

    5.1.4 Dataset Preparation : Colaborative Filtering

This step separates customers into repeat buyers (those with more than one purchase) and one-time buyers. This is important for distinguishing between different customer behavior patterns

In [41]:
df_collaborative = df_olist[df_olist['customer_unique_id'].isin(repeat_customers_ids)]

# 1. Dataset for collaborative filtering
df_collaborative = df_collaborative.groupby(['customer_unique_id', 'product_id'])['review_score'].agg('mean').reset_index()

df_collaborative = df_collaborative.rename({'review_score':'rating'}, axis=1)

(7846, 27)

In [ ]:
df_collaborative.shape

##### **5.2 Content-Based Filtering**

    5.2.1 Similiarity Computation

In [14]:
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(df_content['product_category_name'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    5.2.2 Recommendation Functions

In [15]:
def get_recommendations(product_id, df_content, cosine_sim, top_n=5):
    
    idx = df_content[df_content['product_id'] == product_id].index[0]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_scores = sim_scores[1:top_n+1]  # Skip the item itself

    # Extract product IDs and categories
    recommended_indices = [i[0] for i in sim_scores]
    
    recommended_products = df_content.iloc[recommended_indices][['product_id', 'product_category_name']]

    return recommended_products
     

    5.2.3. Recommendation Results

In [16]:
product_id = 'P20845'

recommendations = get_recommendations(product_id, df_content, cosine_sim)

display(recommendations)

,product_id,product_category_name
2,P23317,office_furniture
3,P20373,office_furniture
42,P17672,office_furniture
72,P21112,office_furniture
292,P29007,office_furniture


    5.2.4 Content-Based Filtering Implementation

In [37]:
def get_user_purchased_products(user_id, df_all):
    # Filter df_all to get products bought by the specified user
    user_purchases = df_all[df_all['customer_unique_id'] == user_id]

    # Extract the product IDs of the products purchased by the user
    purchased_product_ids = user_purchases['product_id'].unique()

    return purchased_product_ids


In [44]:
# Content Based Recommendation System Implementation
user_id =   'C48272'
get_product_id = get_user_purchased_products(user_id,df_olist)
product_id_str = get_product_id[0]

recommendations = get_recommendations(product_id_str, df_content, cosine_sim, top_n=5)

display(get_product_id)
display(recommendations)


array(['P20845'], dtype=object)

,product_id,product_category_name
2,P23317,office_furniture
3,P20373,office_furniture
42,P17672,office_furniture
72,P21112,office_furniture
292,P29007,office_furniture


##### **5.3. Collaborative Filtering**

In [ ]:
null = df_collaborative.isnull().sum()
null

    5.3.1 Data Scaling and Loading

This step scales the ratings and prepares the dataset for training by defining the rating scale and loading it into a dataset object.

In [ ]:
scaler = (df_collaborative.rating.min(), df_collaborative.rating.max())

reader = Reader(rating_scale=scaler)

data = Dataset.load_from_df(df_collaborative[['customer_unique_id', 'product_id', 'rating']], reader)


    5.3.2 Data Splitting 

This step shuffles the data and splits it into training and test sets, ensuring an unbiased evaluation of the models.

In [ ]:
import random

random.seed(42)
all_collab = data.raw_ratings
random.shuffle(all_collab)

# Split data into train and test sets
threshold = int(0.8 * len(all_collab))
train_raw_collab = all_collab[:threshold]
test_raw_collab = all_collab[threshold:]

In [ ]:
listed = [all_collab, train_raw_collab, test_raw_collab]
names = ['all_collab', 'train_raw_ratings', 'test_raw_ratings']

for i, lis in enumerate(listed):
    count = len(lis)
    print(f"Shape of {names[i]}: {count}")

In [ ]:
# Update the dataset with train data
data.raw_ratings = train_raw_collab

    5.3.3 Model Evaluation

This step defines various recommendation models, performs cross-validation, trains them, and calculates their performance metrics (RMSE) on both training and test sets.

    5.3.3.a Define Model

In [ ]:
models = {
    'NormalPredictor': NormalPredictor(),
    'SVD': SVD(n_factors=30, n_epochs=25, biased=True, lr_all=0.00004, reg_all=0.4, random_state=47),
    'SVDpp': SVDpp(n_factors=50, n_epochs=20, lr_all=0.00008, reg_all=0.4, random_state=47),
    'NMF': NMF(n_factors=20, n_epochs=30, lr_bu=0.0000001, lr_bi=0.0000001, reg_pu=5, reg_qi=5, biased=True, random_state=47),
    'KNNBasic': KNNBasic(sim_options={'name': 'cosine', 'user_based': False}, random_state=47),
    'CoClustering': CoClustering(n_cltr_u=8, n_cltr_i=8, n_epochs=30, random_state=47),
    'SlopeOne': SlopeOne()
}

    5.3.3.b Evaluate Model Performance

In [ ]:

results = {}

for name, model in models.items():
    print(f"Evaluating {name}...")
    
    # Cross-validation
    cv_result = cross_validate(model, data, measures=['RMSE'], cv=5, verbose=False, n_jobs=2)
    
    # Train model
    trainset = data.build_full_trainset()
    model.fit(trainset)
    
    # Calculate RMSE for training set
    train_pred = model.test(trainset.build_testset())
    train_rmse = accuracy.rmse(train_pred, verbose=False)
    
    # Calculate RMSE for test set
    testset = data.construct_testset(test_raw_collab)
    test_pred = model.test(testset)
    test_rmse = accuracy.rmse(test_pred, verbose=False)
    
    results[name] = [train_rmse, test_rmse]

# Convert results to DataFrame
df_comparison = pd.DataFrame(results, index=['RMSE Train', 'RMSE Test']).T
print(df_comparison)


    5.3.4 Hyperparameter Tuning

In [ ]:
param_grid = {'n_factors': [25, 50], 'n_epochs': [30, 50], 'lr_all': [0.005, 0.01], 'reg_all': [0.02, 0.1]}
gs_svdpp = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=5)
gs_svdpp.fit(data)

best_score_svdpp = gs_svdpp.best_score['rmse']
best_param_svdpp = gs_svdpp.best_params['rmse']

print(f'Best score: {best_score_svdpp}')
print(f'Best parameter: {best_param_svdpp}')

# Create and fit the best model
best_model_svdpp = SVDpp(**best_param_svdpp)

best_model_svdpp.fit(data.build_full_trainset())


##### **5.4 Hybrid Recommendation System**

In [ ]:
df_content.columns

In [ ]:
df_collaborative.columns

In [ ]:
reader = Reader(rating_scale=(1, 5))  # Adjust rating scale if needed
data = Dataset.load_from_df(df_collaborative[['customer_unique_id', 'product_id', 'rating']], reader)

trainset = data.build_full_trainset()
svdpp = SVDpp()
svdpp.fit(trainset)


In [ ]:
svdpp_model = best_model_svdpp

In [ ]:
def hybrid_recommendation_system(user_id, df_hybrid, df_content, cosine_sim, svdpp_model, top_n=5):
    # Step 1: Content-Based Recommendation
    get_product_id = get_user_purchased_products(user_id, df_hybrid)
    content_based_recommendations = []

    if get_product_id:  # Ensure user has purchased products
        product_id_str = get_product_id[0]  # Get the first product ID from the list
        content_based_recommendations = get_recommendations(product_id_str, df_content, cosine_sim, top_n=top_n)

    # Step 2: Collaborative Filtering - Get top 2 recommended categories
    user_rated_items = df_hybrid[df_hybrid['customer_unique_id'] == user_id]['product_id'].values

    category_predictions = []
    for product_id in df_content['product_id'].unique():
        if product_id not in user_rated_items:
            pred = svdpp_model.predict(user_id, product_id)
            category = df_content[df_content['product_id'] == product_id]['product_category_name'].values[0]
            category_predictions.append((category, pred.est))

    # Sort and select top 2 categories based on the prediction score
    top_categories = sorted(category_predictions, key=lambda x: x[1], reverse=True)[:2]
    top_categories = [category for category, _ in top_categories]

    # Step 3: Cascade into Content-Based for top 2 categories
    cascade_recommendations = []
    for category in top_categories:
        # Filter products by category
        category_products = df_content[df_content['product_category_name'] == category]['product_id'].values
        for product_id in category_products[:2]:  # Select 2 products for each category
            recs = get_recommendations(product_id, df_content, cosine_sim, top_n=top_n)
            cascade_recommendations.extend(recs[['product_id', 'product_category_name']].values.tolist())

    return {
        "content_based_recommendations": content_based_recommendations,
        "collaborative_filtering_recommendations": cascade_recommendations
    }


In [ ]:
user_id = '861eff4711a542e4b93843c6dd7febb0'

# Call the hybrid recommendation system function
recommendations = hybrid_recommendation_system(user_id, df_hybrid, df_content, cosine_sim, svdpp_model)



In [ ]:
def print_recommendations(recommendations):
    # Print Content-Based Recommendations
    print("Similar Items:\n")
    for index, row in recommendations["content_based_recommendations"].iterrows():
        print(f"Product ID: {row['product_id']}, Category: {row['product_category_name']}")

    print("\nYou might also like:\n")
    for rec in recommendations["collaborative_filtering_recommendations"]:
        print(f"Product ID: {rec[0]}, Category: {rec[1]}")

# Call the function to print recommendations
print_recommendations(recommendations)

##### **5.5 Recommendation System For New User**

In a cold-start scenario, where a new user has just joined the platform and there is no prior interaction data, our recommendation system will focus on suggesting popular items. These items are selected based on metrics such as the most purchased, most reviewed, and those trending in the user's geographic area. This approach ensures that new users receive relevant and appealing recommendations even without a personalized history.


    5.5.1 Popularity-Based Recommendations

In the initial stages, the recommendation system leverages a popularity-based approach to provide immediate suggestions. This method selects items based on various popularity metrics:

    5.5.1.a Most Purchased Items

Items with the highest purchase frequency across the platform.

In [26]:
def find_popular_items(data, n_recs):
    top_n_items = data.product_id.value_counts().sort_values(ascending=False)[:n_recs].index
    return list(top_n_items)

    5.5.1.b Popular Items in User's Area

Items that are trending within the user's geographic location, which can help provide contextually relevant suggestions.

In [27]:
def popular_in_your_area(data, state, n_recs):
    location_df = data[data.customer_state == state]
    top_n_items = location_df.product_id.value_counts().sort_values(ascending=False)[:n_recs].index
    return list(top_n_items)

    5.5.2 Implementation of Recomendation Functions

In [28]:
def first_time_recommender(data, uid, n_recs):
    hot_items = find_popular_items(data, n_recs)
    state = data[data.customer_unique_id==uid].customer_state.max()
    popular_in_area = popular_in_your_area(data, state, n_recs)

    print(f"Hot items you might like:\n {hot_items}\n")
    print(f"Popular items in your area:\n {popular_in_area}")

    recommendation = {'Hot Items': hot_items, 'Area': popular_in_area}

    return recommendation

In [29]:
# Example Recommendation
recommendation = first_time_recommender(df_hybrid, 'C48272', 3)


Hot items you might like:
 ['P21247', 'P18956']

Popular items in your area:
 ['P21247', 'P18956']


{'Hot Items': ['P21247', 'P18956'], 'Area': ['P21247', 'P18956']}

##### **5.6 Batch Processing For Unseen Data**

Batch processing allows us to efficiently generate recommendations for a large set of users. Here, we perform batch processing for unseen data after applying collaborative filtering. The following steps outline the process in a structured way.

    5.6.1 Second Transaction with same Categories

    5.6.1.1 Generate Predictions for Each User

In [ ]:
# 1. Extract Unique User IDs
user_ids = repeat_buyers_same_category['customer_unique_id'].unique()

# 2.  Initialize Storage for Recommendations
top_3_recommendations = []

In [ ]:
for user_id in user_ids:
    
    predictions_list = []
    items = repeat_buyers_same_category['product_id'].unique()

    for item_id in items:
        prediction = svdpp.predict(user_id, item_id)
        product_category = repeat_buyers_same_category[
            repeat_buyers_same_category['product_id'] == item_id]['product_category_name'].values[0]
        predictions_list.append({
            'user_id': user_id,
            'item_id': item_id,
            'product_category': product_category,
            'estimated_rating': prediction.est
        })
        
    df_user_predictions = pd.DataFrame(predictions_list)


    5.6.1.2  Calculate Average Estimated Ratings

In [ ]:
df_avg_ratings = df_user_predictions.groupby('product_category')['estimated_rating'].mean().reset_index()

df_top_2 = df_avg_ratings.sort_values(by='estimated_rating', ascending=False).head(2)

own_category = repeat_buyers_same_category[
    repeat_buyers_same_category['customer_unique_id'] == user_id].iloc[-1]['product_category_name']
top_3_categories = [own_category] + df_top_2['product_category'].tolist()


    5.6.1.3 Merge Recommendations with Transaction Data

In [ ]:
df_top_3_recommendations = pd.DataFrame(top_3_recommendations)

merged_df = pd.merge(repeat_buyers_same_category, df_top_3_recommendations, left_on='customer_unique_id', right_on='user_id')

    5.6.1.4 Check Recommendation Accuracy

In [ ]:
merged_df['is_in_top_3'] = merged_df.apply(
    lambda row: row['product_category_name'] in row['top_3_categories'], axis=1
)
percentage_existing_prediction = merged_df['is_in_top_3'].mean() * 100
print(f"Percentage in top 3 recommendations: {percentage_existing_prediction:.2f}%")

    5.6.1.5 Compare Product Categories Between Transactions

In [ ]:
repeat_buyers_same_category = repeat_buyers_same_category.sort_values(by=['customer_unique_id', 'order_id'])
df_comparison = repeat_buyers_same_category.groupby('customer_unique_id').apply(
    lambda x: x.iloc[0]['product_category_name'] == x.iloc[1]['product_category_name']
).reset_index()
df_comparison.columns = ['customer_unique_id', 'same_category']
percentage_same_category = df_comparison['same_category'].mean() * 100
print(f"Percentage with the same category for both transactions: {percentage_same_category:.2f}%")


    5.6.2 Second Transcation with Different Categories

    5.6.2.1 Generate Predictions for Each User

In [ ]:
# 1. Extract Unique User IDs
user_ids = repeat_buyers_diff_category['customer_unique_id'].unique()

# 2.  Initialize Storage for Recommendations
top_3_recommendations2 = []

In [ ]:
for user_id in user_ids:
    predictions_list = []

    # Get the list of all items (products) in the dataset
    items = repeat_buyers_diff_category['product_id'].unique()

    # Predict the rating for each item for the current user
    for item_id in items:
        prediction = svdpp.predict(user_id, item_id)
        product_category = repeat_buyers_diff_category[
            repeat_buyers_diff_category['product_id'] == item_id]['product_category_name'].values[0]

        # Append the prediction details to the list
        predictions_list.append({
            'user_id': user_id,
            'item_id': item_id,
            'product_category': product_category,
            'estimated_rating': prediction.est
        })

    # Convert the predictions list to a DataFrame
    df_user_predictions2 = pd.DataFrame(predictions_list)


    5.6.2.2  Calculate Average Estimated Ratings


In [ ]:
df_avg_ratings2 = df_user_predictions2.groupby('product_category')['estimated_rating'].mean().reset_index()

df_top2 = df_avg_ratings2.sort_values(by='estimated_rating', ascending=False).head(2)

own_category2 = repeat_buyers_diff_category[
    repeat_buyers_diff_category['customer_unique_id'] == user_id].iloc[-1]['product_category_name']
top_3_categories2 = [own_category2] + df_top2['product_category'].tolist()


    5.6.2.3 Merge Recommendations with Transaction Data

In [ ]:
top_3_recommendations2.append({
    'user_id': user_id,
    'top_3_categories': top_3_categories2
})

df_top_3_recommendations2 = pd.DataFrame(top_3_recommendations2)


merged_df2 = pd.merge(repeat_buyers_diff_category, df_top_3_recommendations2, left_on='customer_unique_id', right_on='user_id')


    5.6.2.4 Check Recommendation Accuracy

In [ ]:
merged_df2['is_in_top_3'] = merged_df2.apply(
    lambda row: row['product_category_name'] in row['top_3_categories'], axis=1
)

percentage_existing_prediction2 = merged_df2['is_in_top_3'].mean() * 100
print(f"The percentage of product categories in the dataset that are in the top 3 recommendations is: {percentage_existing_prediction2:.2f}%")


#### **6. Conclusion And Recommendation**

##### **6.1 Conclusion**



#####**6.1. Conclusion** 

1. analysis Result of factors causing the low customer retention rate based on customer behavior as follows :

1.1. The top three product preferences for repeat buyers are bed_bath_table, furniture, and sport_leisure, while the top three for one-time buyers are bed_bath_table, health_beauty, and sports_leisure. There is a noticeable difference where one-time buyers focus more on beauty and health-related items, which could be a good opportunity for personalized offers.

1.2. The largest purchase distribution for both one-time buyers and repeat buyers is concentrated in three cities: Sao Paulo, Rio de Janeiro, and Belo Horizonte. The top three regions are SP (Sao Paulo), RJ (Rio de Janeiro), and MG (Minas Gerais). Analysis shows that these cities and regions also have the highest customer and seller distribution and are among the three largest cities in Brazil, making them a potential focus for business growth through targeted promotional strategies.

1.3. Customer satisfaction analysis for both one-time and repeat buyers shows good scores, with satisfaction levels of 4 & 5 above 70%. However, the analysis also reveals high dissatisfaction scores of 1 & 2, exceeding 10% for both groups. One of the factors affecting these scores is the timeliness of the shipping process, which contributes to the 1 & 2 satisfaction scores but only accounts for about 30%. The remaining analysis from customer comments indicates that over 70% of customer dissatisfaction is due to issues such as incorrect deliveries, damaged goods, undelivered items, and errors in shipment quantity. This is a key area that needs improvement to enhance customer satisfaction, increase customer retention, and positively impact future business growth.

1.4. There is no significant difference in the choice of payment methods and payment installments between one-time buyers and repeat buyers. However, the analysis of purchase frequency by day shows that one-time buyers tend to make more purchases on Mondays, while repeat buyers tend to buy more on Tuesdays. Additionally, based on time, one-time buyers make most of their purchases between 10 AM and 4 PM, while repeat customers peak at 9 AM and 2 PM. Implementing business strategies based on these findings can help Olist improve the effectiveness of marketing campaigns and personalize the shopping experience.

1.5. Regarding the contribution to the company’s revenue, one-time buyers, who make up 97% of the total customer base, contribute 94% of the revenue, while repeat customers, who only make up 3% of the customer base, contribute nearly twice as much, with 5.5% of the revenue.


##### **6.2 Recommendation**